In [1]:
!pip install vk_api

In [2]:
import vk_api

In [3]:
vkApiSession= vk_api.VkApi(token="vk1.a.vW70ZdDvxtVK9YAW98sP6zThPn7FSkyYusK8txhSG3mSl3nuuI-POJXK5BuZeeW-NsV3EePdQRpIrv_WURQzqt-hgAqoOikDT02nYAUXwM84lJ3CFaWiHEniPzZZRM8b7ydA2GMv2icQbe9eks9CPapfNSUtxoemSoCwXHlK3oSf1jSgRBtnz6jZAxVBRjU8gYesb-1MF1ItK4iVe9xclg")

vk = vkApiSession.get_api()

In [4]:
posts = vk.wall.get(owner_id=-20629724, count=100)['items'][10]
posts

{'inner_type': 'wall_wallpost',
 'ads_easy_promote': {'type': 2,
  'text': 'Что-то пошло не так.',
  'label_text': '',
  'button_text': '',
  'is_ad_not_easy': False},
 'comments': {'can_post': 1, 'count': 5, 'groups_can_post': True},
 'marked_as_ads': 0,
 'activity': {'comments': [{'id': 1601444,
    'from_id': 49637527,
    'date': 1734629277,
    'text': 'Покупайте биткоин глупцы',
    'post_id': 1601435,
    'owner_id': -20629724,
    'parents_stack': [],
    'likes': {'can_like': 1,
     'count': 1,
     'user_likes': 0,
     'can_like_by_group': 0,
     'group_liked': False},
    'thread': {'count': 0}}],
  'post_author_id': -20629724,
  'type': 'comments',
  'discriminator': 'reply'},
 'hash': '8czCTaJTa1WAeNQvVk4VAJLNiFOP',
 'type': 'post',
 'push_subscription': {'is_subscribed': False},
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1734624051,
    'id': 457329800,
    'owner_id': -20629724,
    'access_key': '7b38cf3be46ceaecbb',
    'post_id': 16

In [27]:
import csv
import pandas as pd

def scrape_wall_data(token, owner_id, count=10):
    try: # блок try-except для обработки возможных ошибок при взаимодействии с API ВКонтакте.
        vk_session = vk_api.VkApi(token=token) #выполянем запрос по токену
        vk = vk_session.get_api() #получаем объект API, с помощью которого будут выполняться запросы к серверам ВКонтакте.
       
        # Метод wall.get
        posts = vk.wall.get(owner_id=owner_id, count=count)['items'] # записываем записи со стены сообщества в переменную posts

        # Хранилище для данных
        scraped_data = [] # пустой список для добавления данных с каждой записи при иритации 

        # Парсинг
        for post in posts:
            post_data = {
                'post_id': post['id'],
                'text': post['text'],
                'date': post['date'],
                'likes': post['likes'],
            }
            scraped_data.append(post_data)

        return scraped_data

    except vk_api.exceptions.ApiError as e:
        print(f"Error accessing VK API: {e}")
        return []

In [31]:
def save_to_csv(data, file_name='vk_wall_posts.csv'): #сохраним наши данные в файл формата csv
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['post_id', 'text', 'date','likes'])
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [34]:
import pandas as pd

In [69]:
def stats(data):
    df = pd.DataFrame(data)
    print(f'Посмортим на общую информацию: \n {df.describe()}')
    print('\n')
    print(df.info())
    print('\n')
    print(f'Статистика по первым пяти постам\n{df.head()}')
    pass

In [70]:
def main():
    # Введите ваш токен VK API
    token = "vk1.a.vW70ZdDvxtVK9YAW98sP6zThPn7FSkyYusK8txhSG3mSl3nuuI-POJXK5BuZeeW-NsV3EePdQRpIrv_WURQzqt-hgAqoOikDT02nYAUXwM84lJ3CFaWiHEniPzZZRM8b7ydA2GMv2icQbe9eks9CPapfNSUtxoemSoCwXHlK3oSf1jSgRBtnz6jZAxVBRjU8gYesb-1MF1ItK4iVe9xclg"

    # Введите ID сообщества
    owner_id = -299

    # Введите количество публикаций для скрейпинга
    num_posts = 20

    wall_data = scrape_wall_data(token, owner_id, num_posts)
    print('Посмотрим на полученные данные:')
    print(wall_data)
    print('\n')
    if wall_data:
        save_to_csv(wall_data)

        stats(wall_data)
    else:
        print("No data")

if __name__ == "__main__":
    main()

Посмотрим на полученные данные:
[{'post_id': 20335, 'text': 'Сегодня в Питерской Вышке началась зимняя сессия!\n\nВерьте в себя, свои силы и знания — и не забывайте отдыхать в перерывах между экзаменами. Всё получится 💪', 'date': 1734679800, 'likes': {'can_like': 1, 'count': 34, 'user_likes': 0, 'can_publish': 1, 'repost_disabled': False}}, {'post_id': 20329, 'text': '☃ Подводим итоги новогоднего розыгрыша! Подарки от Питерской Вышки получают: \n\n— [id436502976|Денис Кузьмин];\n— [id764354|Ульяна Галеева];\n— [id437049318|Илья Кириллов];\n— [id199732168|Лия Гимадиева];\n— [id327038769|Дмитрий Юн].\n\nПоздравляем победителей! Вы можете забрать подарки в корпусе на Кантемировской, за подробностями напишите в личные сообщения группы. \n\nБлагодарим всех, кто участвовал в конкурсе и поделился хорошими событиями в комментариях! Очень радостно, что у многих они связаны с Питерской Вышкой 💙', 'date': 1734608011, 'likes': {'can_like': 1, 'count': 18, 'user_likes': 0, 'can_publish': 1, 'repost